In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [406]:
##### 파일 불러오기 #####
df = pd.read_csv("C:\\Users\\asus\\BigContest\\files\\data_preprocessing\\1_food_waste_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'waste_cnt', 'waste_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,waste_cnt,waste_amt
base_date,,,,
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850


In [407]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'waste_amt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,waste_cnt
base_date,,
2019-11-30,50110250,620
2019-12-31,50110250,8178
2020-01-31,50110250,13042
2020-02-29,50110250,12264
2020-03-31,50110250,14316
...,...,...
2021-02-28,50130620,4563
2021-03-31,50130620,4920
2021-04-30,50130620,5256


In [408]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130610  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,waste_cnt
base_date,
2018-01-31,17791
2018-02-28,15809
2018-03-31,17920
2018-04-30,17538
2018-05-31,19583
2018-06-30,20601
2018-07-31,24565
2018-08-31,22807
2018-09-30,20540


In [409]:
# series 형으로 변환
sub_df.to_dict('series')

{'waste_cnt': base_date
 2018-01-31    17791
 2018-02-28    15809
 2018-03-31    17920
 2018-04-30    17538
 2018-05-31    19583
 2018-06-30    20601
 2018-07-31    24565
 2018-08-31    22807
 2018-09-30    20540
 2018-10-31    18641
 2018-11-30    16869
 2018-12-31    15818
 2019-01-31    16140
 2019-02-28    13280
 2019-03-31    10960
 2019-04-30    10898
 2019-05-31    16710
 2019-06-30    26735
 2019-07-31    28340
 2019-08-31    28978
 2019-09-30    26520
 2019-10-31    25393
 2019-11-30    23433
 2019-12-31    22029
 2020-01-31    22288
 2020-02-29    20652
 2020-03-31    23026
 2020-04-30    22017
 2020-05-31    25448
 2020-06-30    26003
 2020-07-31    26855
 2020-08-31    29034
 2020-09-30    25933
 2020-10-31    24234
 2020-11-30    21394
 2020-12-31    20924
 2021-01-31    20185
 2021-02-28    19162
 2021-03-31    22121
 2021-04-30    23495
 2021-05-31    27174
 2021-06-30    28002
 Name: waste_cnt, dtype: int64}

# ARIMA

In [410]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [411]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -2.951814
p-value : 0.039646
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [412]:
##### 이건 설치하는 거라 처음 돌릴 때 한 번만 주석 풀어서 설치하고 그 이후로는 필요없음! #####

#!pip install pmdarima

In [413]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=832.571, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=798.992, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.06 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.04 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=772.692, Time=0.18 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=784.665, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=781.026, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=773.227, Time=0.03 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=774.677, Time=0.05 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=776.557, Time=0.07 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=775.701, Time=0.02 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=777.708, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=774.997, Time=0.07 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=775.764, Time=0.10 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept   :

ARIMA(order=(0, 0, 5), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [414]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.waste_cnt.values, order=(0,0,5))     ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 5)   Log Likelihood                -379.001
Method:                       css-mle   S.D. of innovations           1850.535
Date:                Fri, 10 Sep 2021   AIC                            772.002
Time:                        15:46:38   BIC                            784.166
Sample:                             0   HQIC                           776.460
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.196e+04   1594.400     13.773      0.000    1.88e+04    2.51e+04
ma.L1.y        1.3478      0.160      8.428      0.000       1.034       1.661
ma.L2.y        1.5660      0.291      5.376      0.0

# 예측하기

In [415]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([28407.51728846, 21959.54863685]), array([1850.53524215, 3105.65771124]), array([[24780.53486173, 32034.4997152 ],
       [15872.57137451, 28046.52589919]]))
